## **Operations Workshop #1: Geospatial Crash Analysis in Python**

In this workshop, you will learn how to import, manage, and analyze crash data through a geospatial lens. 

#### **Important note**
You can always run jupyter cells by clicking Shift + Enter or Ctrl/Cmd + Enter

### **0: What is Python? Why do we use it for Analysis?**

Python is a highly versatile *programming language* widely used in thousands of applications both in the academic and professional sphere. It can be used to make anything from your favorite app's back end to ChatGPT. 

Python's biggest advantage over its competitors is that it is *open-source*, meaning it has thousands upon thousands of user-created *libraries* (also called *packages*) that *anyone* can use in their code. For the scope of this workshop, we're only going to focus on a few specific packages that help us with geospatial analysis.

#### **0.1: What is GeoSpatial Analysis in python?**

*GeoSpatial Analysis* is, to put it simply, exactly what it sounds like; using Geospatially tagged data to make inferences about other data. Geospatial data in python has a specific structure, and nearly all geospatial data pipelines are based on the package `shapely`, which is notoriously useful for procuring geometries. One of the most widely used python packages, `geopandas`, is an extension of both `shapely` and what is probably the single most used python package: `pandas`. `pandas` is one of the most powerful tools you will come across in any kind of Data Analysis, Data Science, or any other task involving data (it can even be used for budgeting!). 

In transportation fields, Geospatial analyis is absolutely critical to understand the problems at play in a given study area. In this workshop, we are going to leverage Geospatial analysis tools to perform a common but extremely important task: identifying dangerous intersections. 

The "Geospatial" element of this type of work comes into play when particular data about roads is unavailable. In other words, oftentimes when we don't know something about a certain dataset, we like to merge it with other datasets to gain additional insight -- like a missing puzzle piece. With geotagged data, we can use map projections and geometries to gain *spatial* insights about our data -- like understanding where crashes tend to happen, and what kind of crashes tend to happen. 

### **1: Imports and Preprocessing**

#### **1.1: Importing Packages**

You will notice in the first code block that we import 6 packages. Importing is necessary to tell python what different functions mean, especially when multiple packages might have functions that are named the same. (In the event you need both packages, you can specify like `package1.function` and `package2.function` in your code). 

You might notice that we are *renaming* packages. It is tedious to write `geopandas.GeoDataFrame` every time we need to access that method, so we create an *alias*, so that we can write `gpd.GeoDataFrame` (it's only a bit shorter, but it adds up!). 

We already talked about `geopandas`, `pandas`, and `shapely`, but what are the others?

- `numpy`: A package used for optimized numerical computation. `numpy` contains methods for arrays, quick operations over large sets of data (like `np.mean`, `np.std`, and `np.argmin`), and nesting actions. 
- `networkx`: A package used for graph and network-theory based computation. `osmnx` depends on `networkx`, so it is important to understand both when using them in practice. When we talk about graphs and networks, we mean *topological graphs*, which is a way of interpreting systems in terms of nodes (vertices) and edges (arcs) between them. In transportation, graphs are an incredible way of understanding our transportation networks. 

A common use of graph theory is when you get directions on google maps. The app is using a graph representation of a city's road network to compute the best way for you to get to your destination. 

- `osmnx`: A package that builds `networkx` objects but with a geospatial flare. The name comes from OSM, or OpenStreetMap, one of the most well-known open source mapping tools. `osmnx` also works closely with `geopandas` to allow us to move from `networkx` data structures to `pandas`-native datastructures like DataFrames. 

- `folium`: A package used for interactive mapping through leaflet, which is an API that also calls upon OpenStreetMap. `folium` allows us to view our data on a map right in the notebook, without having to export to ArcGIS or some other mapping tool. 




You don't need to do anything to this cell; here we are just explaining the packages being imported. 

In [ ]:
# Ignore this cell; it makes sure everything is installed

%pip install -q geopandas osmnx networkx matplotlib mapclassify pandas numpy folium otter-assign

SUBMISSION_BASENAME = "ops_hw1"

In [ ]:
# Handle Imports. Don't edit this cell

import geopandas as gpd
import osmnx as ox
import networkx as nx
import pandas as pd
import numpy as np
import folium

#### **1.2 Importing the Data**

Now that python can understand our commands, we need to get some data to work with. For our crash data, we are going to use TIMS (Transportation Injury Mapping System), a UC Berkeley resource to access the California Statewide Integrated Traffic Records System (SWITRS). 

1. Go to https://tims.berkeley.edu
2. Create an Account with your Berkeley email. 
3. From the home page, click the 'Analysis & Visualizations' dropdown. 
4. Select "SWITRS Query & Map" 

You should now see a dialogue page where you can select the data you need. 

1. Set the start data to January 1st, 2015 (01/01/2015). Set the end date to March 31st, 2025 (03/31/2025), the latest date TIMS currently has on record. 
2. Select Alameda County, and in the menu on the right, select "City of Alameda" (may just be called "Alameda").
3. Click "Show Result" in the bottom left. 
4. In the top right, select "Download Raw Data". This will download a `.csv` file of the Crash data. 
5. Drag and Drop the File into your Jupyter Notebook Workspace sidebar, so that you can access it locally. You should see the file in the file explorer on the left side. 

Now that we have the data in the local environment, we can start doing some analysis!

For the first task, we have to import our data. Without data, we can't do any analysis. Here you will find the `pd.read_csv()` function very helpful. Notice how we first reference the *package* (pandas) by calling `pd.`, then we reference the *function* (`read_csv`). This structure is how we call *any* function from any package. 

To call (use) a *function* (also known as a *method*), we first type the *name* of the function, then in parenthesis we put the *parameters*. For example, to import some file `data.csv` we might call this function as: 

<div align="center">

```python
pd.read_csv(r'data.csv')
```
</div>

When reading files, it is **super** important that we use the exact file path *relative to our working directory*. The *working directory* is the path that python will try to access initially, unless you tell it to access somewhere else (for example, your working directory might be `C:/Users/_yourname_/Downloads`, or even just `C:/`). If you have this notebook open in jupyterlab, you don't need to worry about setting the working directory; as long as you drag and dropped the file correctly into the sidebar, python will know where to find it.   

In [ ]:
crashes = pd.read_csv(r'...')   # Add path to crash data CSV file here

Notice the `r` in front of the string. We use this to indicate that this is a "raw string", so that python doesn't try to interpret markdown characters like `\n` or `\t` as meaning newline or tab. 

Now, let's view the data to see what we are working with, using the python-native `display` function. Once again, notice how we call the function by typing its name, then subsequently putting its parameters in parenthesis:

In [ ]:
display(crashes.head(5))

*(Note) The `.head()` command is what we use to tell python to show the first $n$ rows of our dataframe. Another useful command is `.tail()`, which shows the last $n$ rows.*

Wow that's a lot of columns! To see all of the columns, we can display `crashes.columns` to see all of their names. Note that `.columns` is an *attribute* instead of a function, so we don't use parenthesis at the end of it. 

In [ ]:
display(...) # Add code to display the columns of the crashes dataframe here

`display` is super useful, but we also don't *need* to call it to visualize an output. Python has two primary commands for visualizing data or information: `print` and `display`. `print` will attempt to output anything inside its parameters as plain text, usually minimally formatted. `display` will attempt to output anything inside its parameters in its native visualization (like a scrollable DataFrame element). 

##### **Why use them?**
The reason why these exist in python is, simply put, *execution order*. Lines of code execute sequentially in python, so if we don't use `print` or `display` only the very last default display will show up. For example, if we tried to do something like: 

<div align="center">

```python
crashes.head()
crashes.columns
```
</div>

We would *only see the columns*, since they are last in the execution order. However, if we instead did: 

<div align="center">

```python
print(crashes.head())
print(crashes.columns)
```
</div>

We would see both in our output. This is especially useful in debugging.

In [ ]:
crashes.columns # Just run this cell

You might notice that there are columns `PRIMARY_RD`, `SECONDARY_RD`, and `INTERSECTION`. Typically, you might use these for a "quick-and-dirty" analysis, but if we want more information about the roads and intersections where these crashes are occurring, we need to do some of our own geospatial work. 

Now we have to turn this regular old `pd.DataFrame` into a `gpd.GeoDataFrame`. You don't need to edit this cell, just read the code. 

In [ ]:
crashes['geometry'] = gpd.points_from_xy(crashes['POINT_X'], crashes['POINT_Y']) # Create the point geometries
crashes = gpd.GeoDataFrame(crashes, geometry='geometry', crs='EPSG:4326')   # Convert to a GeoDataFrame

display(crashes.head())

You might notice a few things here. First is what we call *slicing*. In `pandas`, there are multiple ways to initialize a new column. Here, we used the simple technique of `DataFrame['new column'] = ...`. You can also use `DataFrame.assign()`. You can only use the former if the column does not exist yet (in older versions, you could use it even if it did, but this led to confusing interpretations, and is now deprecated).

Additionally, you might notice that we set a `geometry` parameter when initializing the GeoDataFrame. GeoDataFrames *always* have a special column called `geometry` that python interprets differently than normal columns. This allows for mapping and spatial referencing, which can't be done as easily with normal columns, and is the default geometric column when working with GeoDataFrames. 

Finally, notice the `crs` parameter. This stands for Coordinate Reference System (CRS), which tells `geopandas` how to interpret the geometries, essentially the map projection to use. When working on a global scale, we need multiple projections, because while one projection might be decent in one city, geometries may be off by a few feet in another. `EPSG:4326` is *Web-Mercator*, meaning we reference geometries in lat/long coordinates. Later, you will see us set `EPSG:2227` which is the NAD-83 California North local projection into feet and inches. 

It is also good practice to check for missing or empty geometries, because they can cause problems down the line. In pandas, we can use `.isna()` on a column to find its NaN values, and in geopandas, we can use `.is_empty` to find empty geometries. 

In [ ]:
display(crashes[crashes['geometry'].is_empty])

Looks like we have a few empty geometries. Some of them have coordinates stored in the `LATITUDE` and `LONGITUDE` columns, so we will *impute* our `POINT_X` and `POINT_Y` columns with them. Unfortunately, we will have to drop the rest. 

*Question 1: Why are these geometries empty? Can you think of 3 reasons?*

*Replace this text with your answer...*

You don't need to edit this cell. 

In [ ]:
crashes['POINT_X'].fillna(crashes['LONGITUDE'], inplace=True)   # Replace the empty POINT_X/POINT_Y values
crashes['POINT_Y'].fillna(crashes['LATITUDE'], inplace=True)

crashes_nona = crashes[crashes['POINT_Y'].notna() & crashes['POINT_X'].notna()]  # Drop any remaining empty geometries

*Note the use of the `inplace` parameter, which allows us to modify the DataFrame in place without needing to set `crashes = crashes[...`*

Now we can visualize our data. 

##### **How do we visualize geospatial data?**
There are multiple ways to visualize geospatial data in python. We can utilize `matplotlib.pyplot`'s (also known as `.plt`), but this presents only a static image. For transportation data, we typically aim to use scrollable dynamic mappings. Here is a good place to use the geopandas `gpd.explore()` method, which is really helpful for visualization. 

When we want to use this function, we can simply output it, like we did with `crashes.columns`, since we only want to see one thing. The general usage of `gpd.explore()` looks like:

<div align="center">

```python
GeoDataFrame.explore()
```
</div>

When we only want to deal with one layer (what we call the spatial visualization of some data sharing similar attributes), `.explore()` will *automatically* center the map and choose tiles for us. However, if you want to control things about the map, you can investigate the other parameters of the function here: https://geopandas.org/en/v1.1.1/docs/reference/api/geopandas.GeoDataFrame.explore.html. <br><br>

The last thing to note is that in some cases, particularly when working with special data structures (like a list, array, DataFrame, or a GeoDataFrame), we don't need to *wrap* the object in the function (depending on the function, of course); instead, when the function is native to the same package or a dependency of that package (like how pandas is a dependency of Geopandas), we can often simply tack the function onto the end of the object (like `gdf.explore()`). Try to visualize the *object* `crashes_nona` in the next cell:

(Hint: if you're having trouble, `crashes` & `crashes_nona` are GeoDataFrames)

In [ ]:
... # Put your code here

This is looking good, however, we should notice that a lot of our crashes are way outside the study area.

*Question 2: Why are these crashes in the dataset? Can you think of some reasons? (Hint: think about roads or highways that might run through or near Alameda)*

*Replace this text with your answer...*

Our next step is to do what is called *clipping*, where we clip our data to some geospatial area. First, we need a bounding polygon of our study area, which we can accomplish with osmnx's `ox.geocode_to_gdf`. Just like before, we are calling from the *library* OSMnx (`osmnx`), the *function* `geocode_to_gdf`. You don't need to figure out how to use it, the next step is implemented for you already. Make sure to try and understand how it is being used:

In [ ]:
ox.settings.use_cache = True # If this cell throws an error, change to False
alameda = ox.geocode_to_gdf("Alameda, Alameda County, California, USA")

Just like before, let's visualize the polygon to see what our city boundary looks like (Hint: you just made the GeoDataFrame `alameda`; it's the same as before):

In [ ]:
... # Put your code here

Now, we can clip our crash data to the bounding polygon to make sure we are only looking at local crashes. Use geopandas's `GeoDataFrame.clip()` method to reassign `crashes_nona` to a clipped version here (you shouldn't need to change much).

In [ ]:
crashes_nona = ....clip(alameda.geometry) # Note how we access the geometry colum here. Fill in the ellipsis

Now let's visualize our cleaned crash dataset. Use `.explore()` to see what `crashes_nona` looks like now. We have updated the GeoDataFrame since we last looked at it, and because we are now downstream in execution order from then, the current version of `crashes_nona` will display, instead of the old version we saw last time. This code should look the same as an earlier cell. 

In [ ]:
... # Put your code here

#### **1.3: Creating our network**

Now that we have our crash data, we need to build a road network that we can tag.

><br>**Why do we need a network?** 
>
>Networks are inherently super useful in transportation operations and research, because of one very specific quality: *routability*. Many mathematicians in the past 60 years have put in countless hours of research into what are called network *traversals*; ways of exploring the nodes (vertices) and edges (segments) of a network. When we represent a city's roads as a network (a.k.a. graph), we can utilize very efficient algorithms to understand the behavior of individuals in our cities. 
>
>Because humans typically don't want to walk, bike, or drive farther than they have to, we represent their journeys as *shortest-path problems*, estimating that the average individual is travelling on the minimum-cost path to their destination (functionally, this is usually *somewhat* true). If you do the second ops workshop, you will learn about these type of problems, and the two main algorithms to solve them, known as *Dijkstra's* and *A\**. 
>
>When we represent hundreds or thousands of journeys as shortest-path problems, we can gain a really critical view of our city's roads; we will be able to understand what segments have the most individuals traveling on them, how many individuals travel that way, and how they will react if certain segments aren't usable anymore (from construction, traffic, or otherwise). 
>
>If you want to learn more about networks and transportation, join ops :) <br><br>

For this exercise, we won't focus so much on the capabilities of networks, but rather use OSM's beautifully geotagged networks to tag all of our crashes to intersections and segments.

This is where `osmnx` comes in handy! Fill in the first argument of the code block to create our network of Alameda (Hint: the first argument in `ox.graph_from_polygon` is the Shapely Polygon you want to bound the network to) (Hint #2: Replace the ellipsis with the GeoDataFrame you want to use):

***Note: This next line of code can take a few seconds; let it finish running before trying to run it again***

In [ ]:
G = ox.graph_from_polygon(....geometry[0], network_type = 'drive') # Fill in the ellipsis

To visualize our network, we will use geopandas to get the nodes and edges dataframes. Note the use of the function `ox.graph_to_gdfs()`; it is an extremely critical function in network analysis, useful for moving back and forth between graphs and GeoDataFrames.

Also notice the *nesting* of the `.explore()` call. Nesting refers to when we stack functions inside of each other to perform their collective capabiltiies on one line instead of multiple. Aside from being less lines of code, nesting also uses less computer memory. If we were to calculate everything externally, we require our computers to remember more information. When we nest function calls, the computer can perform all the actions on one line, only remembering the final output. Don't nest too much, though; your code will become unreadable. 

You don't need to edit this cell.

In [ ]:
nodes, edges = ox.graph_to_gdfs(G, nodes=True, edges=True) # Extract geodataframes
nodes.explore(m=edges.explore(), color='red')   # Nest .explore() calls to visualize multiple layers

Ok, this is looking pretty good! Now, we're going to notice a problem here. When we try to tag crashes to intersections, we will run into one of the most famous problems in Geospatial transportation analysis: the *Dual-Carriageway Problem* (It's not really famous, just a very common and annoying problem).

Dual-Carriageways, also known as divided highways, are frequently represented geometrically as separate parallel one-way segments. This is powerful for interpretation and traversal purposes, but very difficult for analysis, because when these roads meet other roads, they create *two intersections* instead of just one. The structure might look like: 

<div align="center">

```
|
<___ . ___
|
___ . ___>
|

```
</div>

When we tag our crashes to the nearest node, we might tag half the crashes at this intersection to one node, and the other half to the other node, and when we look at the final "worst" nodes, they won't even show up, because each one has been undercounted. 

Try and answer this next question before reading about the solution we will use below, it is good practice for thinking about geospatial data.


*Question 3: Can you think of some ways we might handle this problem? Hint: road segments carry other data; in the maps you have made, hover over them to think about what other information we might be able to use.*

*Replace this text with your answer...*

The easiest solution (and the one we will use here) is to assign a *parent node* to each node (intersection), so that we can just refer to it every time we tag a crash. You don't need to edit this cell, but you should read it to see if you can understand what's happening.

> **Code Breakdown, for Those Interested**
> 
> This code looks super complicated, but we can break it down into parts, for those who are interested. First, we have to *project* the graph into a local CRS, so that we can measure distances in feet and inches instead of web-mercator degrees. The next step is to *consolidate intersections*. This depends on a complicated algorithm from a study done earlier this year, (https://onlinelibrary.wiley.com/doi/10.1111/tgis.70037), but essentially is contracting each multi-node intersection into one intersection. 
>
> We don't want to keep the graph in the consolidated data structure. Instead, in the for loop, we deconstruct the consolidated nodes, which carry an `osmid_original` signature, into the original nodes that they came from. More specifically, we *iterate* through the consolidated nodes, and for each one, we face two cases:
> 1. If `osmid_original` is represented by an integer, it is it's own parent, and no other node has it as a parent; we assign its parent to be itself (this is the `else` case). We add to the `parent_of` dictionary the entry `osmid: parent_osmid`. 
> 2. If `osmid_original` is represented by a list (osmnx stores these as strings), we deconstruct the string into the list, then assign every node in the list to the first node in the list as its parent in the `parent_of` dictionary as the entry `osmid: parent_osmid`. This is the `if` case (Note that `ast.literal_eval` allows us to convert the strings into lists).
>
> Finally, we give each node an attribute for its parent node, which is from the `parent_of` dictionary we modify on every iteration. 

You only need to run the cell below, you don't need to modify it. 

In [ ]:
import ast

Gp = ox.project_graph(G, to_crs=2227)   # The CRS we talked about earlier, this is in feet and inches
Gc = ox.consolidate_intersections(Gp, tolerance=55, rebuild_graph=True, dead_ends=True) # 55 foot tolerance to consolidate

parent_of = {}  # Create dictionary to map nodes to each their parent
for u, data in Gc.nodes(data=True):
    if isinstance(data.get('osmid_original'), str):
        l = ast.literal_eval(data['osmid_original'])    # Convert string representation of list back to list
        for original_osmid in l:
            parent_of[original_osmid] = l[0]    # Set parent as first element of the list
    else:
        parent_of[data.get('osmid_original')] = data.get('osmid_original')  # If there is no parent node, set to itself

nx.set_node_attributes(Gp, parent_of, name="parent")    # Set the node attributes

Now every node has a parent node! This means that once we tag every crash to a node, we will be able to associate it with its parent as well, meaning no undercounting will occur. 

#### **1.4: Tagging the crashes**

Our last order of business is to tie each crash to a node and a parent node. This is the **most critical** step in this process. There are two ways we can go about this, one using geopandas and the other using OSMnx. 
1. We can use geopandas `gpd.sjoin_nearest` method to perform a *spatial join* of each crash and the node dataset. A spatial join can be exact (usually used to check if geometries touch each other), or 'nearest', like in this case, where we might want to find the nearest node within $d$ feet of our crash. 
2. We can use `ox.nearest_nodes`, which find the nearest node *in a graph* to any point. 

The latter will be easier in our case. 

*Question 4: Can you figure out why it might be easier? Look at the code breakdown of the code that creates the parent nodes. Why might it be easier to know the OSM ID of the nearest node, rather than the GeoDataFrame index?* 

*Replace this text with your answer...*

You don't need to edit this cell, but you should definitely try to understand it. *(Note that, rather ironically, we never actually access the geometry of our crashes. In many real-world cases, we won't automatically have an $x$ and $y$ column for our points, and we will have to build them from the geometry)*

In [ ]:
crashes_nona['node'], crashes_nona['dist'] = ox.nearest_nodes(G,    # Find the nearest node to each crash, and the distance
                                                              crashes_nona['POINT_X'], 
                                                              crashes_nona['POINT_Y'], 
                                                              return_dist=True)

crashes_nona['parent'] = crashes_nona['node'].map(nx.get_node_attributes(Gp, 'parent'))  # Map the parent nodes

Visualize the `crashes_nona` dataframe now using `pd.head()`. 

*Question 5: What columns have we added?* *Hint: `.head()` works the same as `.explore()`.*

*Replace this text with your answer...*

In [ ]:
... # Add your code here

### **2: Understanding the Crashes**

We have now successfully geotagged our crashes to intersections in a routable network (meaning a connected graph). Our next step is to start understanding which intersections need the most work. 

We are going to use two very important functions to do this, pandas's `.groupby()` and `.value_counts()`. As a reminder, if you are ever unsure how to use a function, you can make a new cell and type `function?` and run the cell to see its documentation. 

Start by grouping the crashes by their parent intersections. This will tell us precisely what intersections we need to worry most about. 

In [ ]:
top10 = crashes_nona.groupby(...).size().sort_values(ascending=False).head(10)  # Fill in the ellipsis

What we are seeing here is an interesting structure:

```
parent
53098044      22
5050004181    22
53092474      20
53098027      20
53092433      18
...
```

So at node 53098044, there were 22 crashes between 2015 and 2025. However, this means very little to us, as we don't know *where* this is. 

We have a couple ways of resolving this. The first is to search through the edges dataframe for roads with either a `u` or `v` value (these denote the start or end nodes) of our 'dangerous' nodes, then extract their street names, then look at those streets on or map. 

Albeit significantly easier, we can simply map the 10 most dangerous intersections using `.explore()`, and simply zoom to each node and check its OSM ID. To make it visually easier, we could also add in crash numbers as a node attribute, but that is unnecessary for this exercise (it might be useful with thousands of intersections though). <br><br>

To do this, we are going to use an important attribute of a Series: `.index`. We already used an attribute earlier: `.columns`. A *function* must be called, using `()`, but an *attribute* doesn't need parenthesis; it is a fixed piece of information carried by the object. `.index` works for both pandas Series (and geopandas GeoSeries) as well as DataFrames (or GeoDataFrames). `.columns` works *only* for DataFrames, or will return the only column name of a series. If you want to access the values of a Series, the attribute `.values` is useful. 

Before, recall that we were *slicing* our dataframe by just bracketing a value, like a column name (i.e. `df['column']`). Slicing is only allowed like this for column names or exact indices. However, the more general pandas method is `.loc` and `.iloc`. Note that these are not functions, they are *accessors*, which tell the object to find a certain subset of values. Both are ordered by $(\text{index}, \text{column})$, however, `.loc` uses the exact index and exact column name, whereas `.iloc` accesses by the number of indexes before the index, and the number of columns before the column.<br><br>

Now, say, if I wanted to get a *slice* of a dataframe at the *indices* I found from a *.value_counts()* call, to find the rows with the top $n$ in some value, I would write some code like this: 

<div align="center">

```python
values = df.value_counts().head(5)
display(df.loc[values.index])
```
</div>

In the cell below, use the `.index` attribute on the `top10` Series from the previous cell to plot the top 10 intersections by crash number in folium (*Hint: the output of `.value_counts()` looks very similar to the output we created above*):

In [ ]:
crashes_nona.loc[...].explore()

#### **2.1: A Cross-comparison**

Now repeat the code from the last two but for the top 5 crashes instead of the top 10. Go to https://maps.google.com. Zoom into these intersections. 

*Question 4: Do you notice anything interesting about them? Does anything jump out at you as being blatantly problematic?*

In [ ]:

... # Your code here


*Replace this text with your answer...*

#### **2.2 Diving Deeper**

Just knowing what intersections are problematic isn't quite enough for us; it doesn't tell the whole story. Luckily, we have 79 more columns to explore. 

Pick one of the top 5 intersections. First, filter for crashes with that parent. Replace the ellipsis with the 9 or 11 digit OSM ID number of that node. 

In [ ]:
StudyNode = crashes_nona[crashes_nona['parent'] == ...]  # Fill in the ellipsis

Let's first investigate the type of collisions happening at that node. Can you search through the columns we looked at earlier (in `crashes.columns`) to figure out what column this might be? Filter for it by replacing the ellipsis with the exact name of that column (copy-paste is your friend here). 

In [ ]:
crashTypeCounts = StudyNode['...'].value_counts()   # Fill in the ellipsis with the column name
print(crashTypeCounts)
crashTypeCounts.plot(kind='bar')

These letters don't indicate much. Go to https://tims.berkeley.edu/help/SWITRS.php. 

*Question 5: What do the different crash types indicate? What does that tell you about this intersection? What kind of crashes happen most, and how many of those crashes occurred?*

*Replace this text with your answer...*

### **3: Bonus: Time to try it on your own**

Pick a new column from the columns list. Repeat the steps from part **2**. In 3-5 sentences, explain what that column tells you about this intersection, and what kind of takeaways and insights we can gain from that information. 

In [ ]:

... # Put your analysis here


### **4: Submit**

**Make sure to SAVE THE NOTEBOOK first!!**

Run the following cell to create a clickable link to download this notebook. When you are done, submit it through bcourses. 

In [ ]:
from otter import api

api.export_notebook(
    "OpsWorkshop.ipynb",
    exporter_type="html",          # <- this triggers WebPDF exporter
    allow_chromium_download=True,  # downloads Chromium if not bundled
)